In [ ]:
!pip install pandas regex scikit-learn transformers torch google-cloud-storage

In [2]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import numpy as np
import os
from google.colab import auth
from google.cloud import storage
import json
from pandas import testing
import time
import requests
import re
import requests
from bs4 import BeautifulSoup

In [3]:
# Authenticate with GCP
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/steel-climber-398320-634bf855feea.json'

In [4]:
def download_folder(bucket_name, source_folder, destination_dir):
    """Downloads a folder from the bucket."""
    try:
        storage_client = storage.Client('steel-climber-398320')  # Project of the bucket. Must be included
        bucket = storage_client.bucket(bucket_name)

        # List all blobs in the bucket
        blobs = bucket.list_blobs(prefix=source_folder)  # Add prefix to optimize listing

        for blob in blobs:
            if not blob.name.endswith('/'):  # Skip directories
                # Construct the local file path
                local_file_path = os.path.join(destination_dir, blob.name[len(source_folder):])
                # Create local path folders
                os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
                # Download file by file
                blob.download_to_filename(local_file_path)
                # Keep track
                print(f"Blob {blob.name} downloaded to {local_file_path}.")
    except Exception as e:
        print(f"Error occurred: {e}")



In [5]:
#first, scrap the information from medscape

url = 'https://reference.medscape.com/drugs'
response = requests.get(url)
if response.status_code != 200:
    print("Failed to retrieve the main webpage")
response.status_code

soup = BeautifulSoup(response.content, 'html.parser')
category_names = []
category_urls = []
categories = soup.select('ul.classdruglist li a')

all_data = []
for category in categories:
    category_names.append(category.get_text(strip=True))
    category_urls.append(category['href'])

df1 = pd.DataFrame({
    'Category Name': category_names,
    'Category URL': category_urls
})

In [6]:
all_drugs_df = pd.DataFrame()

# loop over the rows in df1
for _, row in df1.iterrows():
    category_name = row['Category Name']
    url = row['Category URL']
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text
        pattern = r'<div class="topic-head" data-id=".*?">(.*?)</div>.*?<a href="(.*?)">(.*?)</a>'
        matches = re.findall(pattern, html_content, re.DOTALL)
        df_drugs = pd.DataFrame(matches, columns=['Class', 'Link', 'Drug'])
        df_drugs['Category'] = category_name
        all_drugs_df = pd.concat([all_drugs_df, df_drugs], ignore_index=True)

    else:
        print(f"Failed to retrieve the webpage for category: {category_name}")

    # sleep for 1seg to avoid overwhelming the server
    time.sleep(1)

In [7]:
#function to extract conditions and doses
def extract_conditions_and_doses(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    tags = soup.find_all(['h3', 'p'])
    condition_doses = {}
    dose_pattern = r'\b\d+(\.\d+)?\s*(mg|g|ml|mL|mcg|L|IU)\b|\bq(Day|Week|Hour|Month)\b'

    current_condition = None
    dose_info = ''

    for tag in tags:
        if tag.name == 'h3':
            # if we have accumulated dose info, save it before moving to the next condition
            if current_condition and dose_info:
                condition_doses[current_condition] = dose_info.strip()
                dose_info = ''

            # update the current condition
            current_condition = tag.get_text().strip()

        elif tag.name == 'p' and current_condition:
            # check if the paragraph contains dosing information
            if re.search(dose_pattern, tag.get_text(), re.IGNORECASE):
                dose_info += tag.get_text().strip() + ' '

    # add the last accumulated dose info
    if current_condition and dose_info:
        condition_doses[current_condition] = dose_info.strip()

    return condition_doses

In [8]:
data = []

# iterate over each row in the all_drugs_df data frame
for index, row in all_drugs_df.iterrows():
    drug_name = row['Drug']
    html_link = row['Link']

    response = requests.get(html_link)
    html_content = response.text

    # extract conditions and doses
    extracted_data = extract_conditions_and_doses(html_content)

    # append the data to the list
    for condition, dose in extracted_data.items():
        # remove comments in parentheses from condition name
        clean_condition = re.sub(r'\(.*?\)', '', condition).strip()
        data.append([drug_name, condition, clean_condition, dose])

# create a new data frame from the list
prescriptions = pd.DataFrame(data, columns=['Drug Name', 'Indication', 'Clean Indication', 'Dosing Information'])


In [9]:
# get icd10 codes embeded
bucket_name = 'embedded_data'
source_folder = "icd_10/"
destination_dir = "/content/data"
download_folder(bucket_name, source_folder, destination_dir)

Blob icd_10/icd10_embed.json downloaded to /content/data/icd10_embed.json.


In [11]:
#get the final_dataframe file to have the icd10 codes and diagnoses being predicted
bucket_name = 'scraped_data_clean'
source_folder = "icd_10_codes_diagnosis/"
destination_dir = "/content/data"
download_folder(bucket_name, source_folder, destination_dir)

#load the dataset
file_path = '/content/data/final_dataframe.csv'
icd_10_diagnosis_used = pd.read_csv(file_path)

Blob icd_10_codes_diagnosis/final_dataframe.csv downloaded to /content/data/final_dataframe.csv.


In [12]:
# function to remove upper cases and strange symbols
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters (keeping spaces)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [13]:
# apply preprocessing to the diagnosis that will be embedded
prescriptions['indication_to_embed'] = prescriptions['Clean Indication'].apply(preprocess_text)

In [14]:
# load tokenizer and model for bioBERT
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
model = BertModel.from_pretrained('dmis-lab/biobert-v1.1')

In [15]:
# function to create the embeddings of the diagnosis and descriptions
def text_to_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    assert isinstance(embedding, np.ndarray)
    return embedding

In [16]:
# get embeddings
prescriptions_embeddings = prescriptions['indication_to_embed'].apply(text_to_embedding)

In [17]:
# create and embeddings column for future use in the dashboard
prescriptions['indication_embedding'] = prescriptions_embeddings

In [18]:
#load the icd_10s already embeded
file_path = '/content/data/icd10_embed.json'
icd10_embeddings = []
with open(file_path, 'r') as file:
    for line in file:
        try:
            icd10_embeddings.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Error decoding JSON from line: {line}")

icd10_embeddings  = pd.DataFrame(icd10_embeddings)

In [19]:
icd_10_diagnosis_used = pd.merge(icd_10_diagnosis_used, icd10_embeddings, left_on = 'Matched_Code', right_on='code', how='left')


In [20]:
diagnosis_embedding_column = icd_10_diagnosis_used['diagnosis_embedding']

# apply the function to each element in the column
processed_arrays = diagnosis_embedding_column.apply(lambda x: np.array(x).reshape(1, -1))

# vertically stack the resulting arrays
icd10_embeddings_array = np.vstack(processed_arrays)

In [21]:
# convert embeddings in prescriptions DataFrame to a matrix
prescription_embeddings_matrix = np.vstack(prescriptions['indication_embedding'].apply(np.array))

In [22]:
# nitialize an empty list to store the diagnosis-prescription pairs
diagnosis_prescription_pairs = []

In [23]:
# Function to find top 5 matching prescriptions for a given diagnosis embedding
def find_top_5_matches(diagnosis_embedding, diagnosis_row):
    # Reshape diagnosis embedding for cosine similarity calculation
    diagnosis_embedding_reshaped = np.array(diagnosis_embedding).reshape(1, -1)

    # Calculate cosine similarity with all prescriptions
    similarity_scores = cosine_similarity(diagnosis_embedding_reshaped, prescription_embeddings_matrix)[0]

    # Get indices of top 5 matches
    top_5_indices = np.argsort(similarity_scores)[-5:]

    # Append pairs to the list, including relevant prescription details
    for index in top_5_indices:
        # Create a dictionary with diagnosis and prescription data
        prescription_data = prescriptions.iloc[index].to_dict()
        pair = {**diagnosis_row, **prescription_data}
        diagnosis_prescription_pairs.append(pair)


In [24]:
# Apply the function to each diagnosis
for _, row in icd_10_diagnosis_used.iterrows():
    find_top_5_matches(row['diagnosis_embedding'], row.to_dict())

# Create a new DataFrame from the pairs
diagnosis_prescription_df = pd.DataFrame(diagnosis_prescription_pairs)

In [25]:
# drop embedding columns
diagnosis_prescription_df = diagnosis_prescription_df.drop(['diagnosis_embedding', 'indication_embedding'], axis=1)


In [26]:
# Convert DataFrame to CSV
diagnosis_prescription_df.to_csv('/content/diagnosis_prescription_before_human_rev.csv', index=False)

# Initialize a client
storage_client = storage.Client()
bucket_name = 'scraped_data_clean'
destination_blob_name = 'prescriptions/diagnosis_prescription_before_human_rev.csv'
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename('diagnosis_prescription_before_human_rev.csv')

The data base is sent to human review and realoaded. Up to this point it is replicable.

In [28]:
#retrive the data base reviewed by a human (ideally, a doctor)
bucket_name = 'scraped_data_clean'
source_folder = "prescriptions/"
destination_dir = "/content/data"
download_folder(bucket_name, source_folder, destination_dir)

reviewed_df = pd.read_csv('/content/data/diagnosis_prescription_human_reviewed.csv', encoding='latin1')

#select the rows that, according to the human reviewer corresponf to drug and the indication
reviewed_df = reviewed_df[reviewed_df['manual_review'] == 'x']

Blob prescriptions/diagnosis_prescription_before_human_rev.csv downloaded to /content/data/diagnosis_prescription_before_human_rev.csv.
Blob prescriptions/diagnosis_prescription_human_reviewed.csv downloaded to /content/data/diagnosis_prescription_human_reviewed.csv.
Blob prescriptions/prescription_database.csv downloaded to /content/data/prescription_database.csv.


In [29]:
reviewed_df = reviewed_df[['Original Code_x', 'Matched_Code', 'description_x', 'Drug Name',
       'Dosing Information']]

In [30]:
#Some final edits to on the strings
reviewed_df['Drug Name'] = reviewed_df['Drug Name'].str.capitalize()
reviewed_df['description_x'] = reviewed_df['description_x'].str.capitalize()

In [ ]:
reviewed_df.to_csv('/content/prescription_database.csv', index=False)

# Initialize a client
storage_client = storage.Client()
bucket_name = 'scraped_data_clean'
destination_blob_name = 'prescriptions/prescription_database.csv'
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename('prescription_database.csv')